In [1]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

# Get all links from Catalog page:

In [68]:
ua = UserAgent()
header = {'User-Agent':str(ua.chrome)}

In [69]:
# Define the URL you want to scrape
url = 'https://www.gucci.com/it/it/ca/women/ready-to-wear-for-women-c-women-readytowear'

In [70]:
# Send an HTTP request to the URL
response = requests.get(url, headers = header)
soup = BeautifulSoup(response.content, 'html.parser')

In [71]:
item_list = soup.select('article a')

In [11]:
for item in item_list: 
    link = 'https://www.gucci.com' + item['href']
    print(link)

https://www.gucci.com/it/it/pr/women/ready-to-wear-for-women/knitwear-for-women/knitted-dresses-for-women/short-stretch-viscose-pleated-dress-p-786401XKD2R9288
https://www.gucci.com/it/it/pr/women/ready-to-wear-for-women/knitwear-for-women/knitted-dresses-for-women/stretch-viscose-polo-dress-p-788864XKD269791
https://www.gucci.com/it/it/pr/women/ready-to-wear-for-women/outerwear-for-women/gg-nylon-jacket-with-web-p-801106Z40399692
https://www.gucci.com/it/it/pr/whats-new/new-in/this-week-women/ready-to-wear-for-women/short-stretch-viscose-pleated-skirt-p-691712XKD279288
https://www.gucci.com/it/it/pr/women/ready-to-wear-for-women/activewear-for-women/technical-jersey-short-with-web-p-785500XJGK69039
https://www.gucci.com/it/it/pr/women/ready-to-wear-for-women/activewear-for-women/short-technical-jersey-dress-with-web-p-785490XJGK69039
https://www.gucci.com/it/it/pr/women/ready-to-wear-for-women/activewear-for-women/jersey-drill-jacket-with-web-p-785483XJGK39799
https://www.gucci.com/it

# Get from specific product page:
- Relevant pictures
- item description

In [176]:
# New Link
link = 'https://www.gucci.com/it/it/pr/women/ready-to-wear-for-women/skirts-for-women/midi-long-skirts-for-women/gg-supreme-oxford-cotton-skirt-p-788904ZAM9M5307'

In [177]:
# Send an HTTP request to the URL
response_1 = requests.get(link, headers = header)
soup_1 = BeautifulSoup(response_1.content, 'html.parser')

In [178]:
list = soup_1.find_all('picture' )


In [186]:
list = soup_1.find_all(attrs={"data-image-size": "small-retina"})

In [188]:
keeping = ['001','002','004','005','012']
# 001: Prod
# 002: Model
# 004: back
# 005: Closeup 1
# 0012: closeup 2

In [209]:
keeping = ['001', '002', '004', '005', '012']

def contains_required_number(line):
    return any(num in line for num in keeping)

In [187]:
i = 0
for item in list:
    i+=1 
    if i == len(list) | contains_required_number(item['srcset']) :
        continue
    else:
        print(item['srcset'] ) 

//media.gucci.com/style/DarkGray_Center_0_0_980x980/1713891716/788904_ZAM9M_5307_002_100_0000_Light-Gonna-in-cotone-Oxford-GG-Supreme.jpg
//media.gucci.com/style/DarkGray_Center_0_0_980x980/1713805269/788904_ZAM9M_5307_001_100_0000_Light-Gonna-in-cotone-Oxford-GG-Supreme.jpg
//media.gucci.com/style/DarkGray_Center_0_0_980x980/1713891716/788904_ZAM9M_5307_002_100_0000_Light-Gonna-in-cotone-Oxford-GG-Supreme.jpg
//media.gucci.com/style/DarkGray_Center_0_0_980x980/1713805271/788904_ZAM9M_5307_003_100_0000_Light-Gonna-in-cotone-Oxford-GG-Supreme.jpg
//media.gucci.com/style/DarkGray_Center_0_0_980x980/1713805271/788904_ZAM9M_5307_004_100_0000_Light-Gonna-in-cotone-Oxford-GG-Supreme.jpg
//media.gucci.com/style/DarkGray_Center_0_0_980x980/1713805272/788904_ZAM9M_5307_005_100_0000_Light-Gonna-in-cotone-Oxford-GG-Supreme.jpg
//media.gucci.com/style/DarkGray_Center_0_0_980x980/1713805273/788904_ZAM9M_5307_007_100_0000_Light-Gonna-in-cotone-Oxford-GG-Supreme.jpg
//media.gucci.com/style/DarkGray_C